In [ ]:
import torch, math
from torch import nn

In [ ]:
class input_embeddings(nn.Module):

  def __init__(self,
               d_model,
               vocab_size):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embeddings = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    return self.embeddings(x) * math.sqrt(self.d_model)

In [ ]:
class positional_encoding(nn.Module):

  def __init__(self,
               d_model,
               seq_len,
               dropout):
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)
    positional_encoding = torch.zeros(seq_len, d_model)
    positions = torch.arange(0, seq_len, dtype= torch.float).unsqueeze(1)
    divide = torch.exp(torch.arange(0, d_model, 2).float()*(-math.log(10000.0/d_model)))
    positional_encoding[:,0::2] = torch.sin(positions*divide)
    positional_encoding[:, 1::2] = torch.cos(positions*divide)
    positional_encoding = positional_encoding.unsqueeze(0)

    self.register_buffer('positional_encoding', positional_encoding)

  def forward(self, x):
    x = x + (self.positional_encoding[:, :x.shape[1], :]).requires_grad_(False)
    return self.dropout(x)

In [ ]:
class layer_normalization(nn.Module):

  def __init__(self, eps=10e-6):
    super().__init__()
    self.eps = eps
    self.alpha = nn.Parameter(torch.ones(1))
    self.bias = nn.Parameter(torch.zeros(1))

  def forward(self, x):
    mean = x.mean(dim=-1, keepdim=True)
    std = x.std(dim=-1, keepdim=True)

    return self.alpha*(x-mean)/(std+self.eps) + self.bias

In [ ]:
class feed_forward_block(nn.Module):

  def __init__(self,
               d_model,
               d_ff,
               dropout):
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)

  def forward(self, x):
    return self.linear_2(self.dropout(self.relu(self.linear_1(x))))

In [ ]:
class multi_head_attention(nn.Module):

  def __init__(self,
               d_model,
               h,
               dropout):
    super().__init__()
    self.d_model = d_model
    self.h = h

    assert d_model%h==0, 'Division Error'

    self.d_k = d_model//h

    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)
    self.w_o = nn.Linear(d_model, d_model)
    self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query,
                  key,
                  value,
                  mask,
                  dropout):
      d_k = query.shape[-1]
      attention_scores = (query@key.transpose(-2,-1))//math.sqrt(d_k)

      if mask is not None:
        attention_scores.masked_fill_(mask == 0, -1e9)

      attention_scores = attention_scores.softmax(dim=-1)

      if dropout is not None:
        attention_scores = dropout(attention_scores)

      return (attention_scores@value), attention_scores

    def forward(self,
                q, k, v,
                mask):
      query = self.w_q(q)
      key = self.w_k(k)
      value = self.v_k(v)

      query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)
      key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)
      value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).tranpose(1,2)

      x, self.attention_scores = multi_head_attention.attention(query, key, value, mask, self.dropout)

      x = x.tranpose(1,2).contiguous().view(x.shape[0], -1, self.h, self.d_k)

      return self.w_o(x)

In [ ]:
class residual_connection(nn.Module):
  def __init__(self, dropout):
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = layer_normalization()

  def forward(self, x, sub_layer):
    return x+self.dropout(sub_layer(self.norm(x)))

In [ ]:
class encoder_block(nn.Module):

  def __init__(self,
               self_attention_block: multi_head_attention,
               feed_forward_block_: feed_forward_block,
               dropout):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_bl = feed_forward_block_
    self.residual_connections = nn.ModuleList([residual_connection(dropout) for _ in range(2)])

  def forward(self, x, src_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))

    x = self.residual_connections[1](x, self.feed_forward_bl)
    return x

In [ ]:
class encoder(nn.Module):
  def __init__(self, layers:nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = layer_normalization()

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
      return self.norm(x)

In [ ]:
class decoder_block(nn.Module):

  def __init__(self,
               self_attention_block: multi_head_attention,
               cross_attention_block: multi_head_attention,
               feed_forward: feed_forward_block,
               dropout):
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward = feed_forward
    self.residual_connections = nn.ModuleList([residual_connection(dropout) for _ in range(3)])

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
    x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
    x = self.residual_connections[2](x, self.feed_forward)
    return x

In [ ]:
class decoder(nn.Module):
  def __init__(self, layers:  nn.ModuleList):
    super().__init__()
    self.layers = layers
    self.norm = layer_normalization()

  def forward(self, x,
              encoder_output,
              src_mask,
              tgt_mask):
    for layer in self.layers:
      x = layer(x, encoder_output, src_mask, tgt_mask)
    return self.norm(x)

In [ ]:
class projection_layer(nn.Module):
  def __init__(self, d_model, vocab_size):
    super().__init__()
    self.project = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    return torch.log_softmax(self.project(x), dim=-1)

In [ ]:
class transformer(nn.Module):

  def __init__(self,
               encoder,
               decoder,
               src_embed,
               tgt_embed,
               src_pos,
               tgt_pos,
               projection_layer):
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output, src_mask, tgt, tgt_mask):
    tgt = self.tgt_embed(tgt)
    tgt = self.tgt_pos(tgt)
    return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

  def project(self, x):
    return self.projection_layer(x)

In [ ]:
def build_transformer(src_vocab_size,
                      tgt_vocab_size,
                      src_seq_len,
                      tgt_seq_len,
                      d_model=512,
                      N=6,
                      h=8, dropout=.1,
                      d_ff=2048):
  src_embed = input_embeddings(d_model, src_vocab_size)
  tgt_embed = input_embeddings(d_model, tgt_vocab_size)
  src_pos = positional_encoding(d_model, src_seq_len, dropout)
  tgt_pos = positional_encoding(d_model, tgt_seq_len, dropout)

  encoder_blocks = []
  for _ in range(N):
    encoder_self_attention = multi_head_attention(d_model, h, dropout)
    feed_forward = feed_forward_block(d_model, d_ff, dropout)
    encoder_block_ = encoder_block(encoder_self_attention, feed_forward, dropout)
    encoder_blocks.append(encoder_block_)

  decoder_blocks = []
  for _ in range(N):
    decoder_self_attention = multi_head_attention(d_model, h, dropout)
    decoder_cross_attention = multi_head_attention(d_model, h, dropout)
    feed_forward = feed_forward_block(d_model, d_ff, dropout)
    decoder_block_ = decoder_block(decoder_self_attention, decoder_cross_attention, feed_forward, dropout)
    decoder_blocks.append(decoder_block_)

  encoder_ = encoder(nn.ModuleList(encoder_blocks))
  decoder_ = decoder(nn.ModuleList(decoder_blocks))
  projection = projection_layer(d_model, tgt_vocab_size)

  transformer_build = transformer(encoder_, decoder_, src_embed, tgt_embed, src_pos, tgt_pos, projection)

  for parameter in transformer_build.parameters():
    if parameter.dim()>1:
      nn.init.xavier_uniform_(parameter)

  return transformer_build

In [ ]:
basic_config = {
    'seq_len': 350,
    'd_model': 512,
    'lang_src': 'en',
    'lang_tgt': 'it',
    'batch_size': 8,
    'num_epochs': 20,
    'lr': 1e-4
}

In [ ]:
transformer = build_transformer(512, 512, 350, 350)